In [14]:
import pennylane as qml
import pennylane.numpy as np

dev = qml.device('default.qubit')

In [15]:
def state_prep(x):
    qml.BasisState(x, wires=[i for i in range(len(x))])

In [16]:
def layer(layer_weights):
    for wire in range((len(layer_weights) - 1) // 2):
        qml.Rot(*layer_weights[2*wire], wires=wire)
        qml.CRot(*layer_weights[wire + 1], [wire, wire + 1])
    
    qml.Rot(*layer_weights[len(layer_weights) - 1], wires=(len(layer_weights) + 1) // 2)
    
    for wires in range(((len(layer_weights) - 1) // 2) - 1):
        wire_actual = ((len(layer_weights) - 1) // 2) - wires - 1
        qml.CNOT([wire_actual, wire_actual - 1])

In [17]:
@qml.qnode(dev)
def circuit(weights, x):
    state_prep(x)

    for layer_weights in weights:
        layer(layer_weights)

    return qml.expval(qml.PauliZ(len(x) - 1))

In [18]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [19]:
def square_loss(labels, predictions):
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

In [20]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [21]:
data = np.loadtxt("data/parity_train_2.txt", dtype=int)
X = np.array(data[:, :-1])
Y = np.array(data[:, -1])
Y = Y * 2 - 1  # shift label from {0, 1} to {-1, 1}

for x,y in zip(X, Y):
    print(f"x = {x}, y = {y}")

x = [0 0 0 0 0 0 0 0 0 0], y = -1
x = [0 0 0 0 0 0 0 0 0 1], y = 1
x = [0 0 0 0 0 0 0 0 1 0], y = 1
x = [0 0 0 0 0 0 0 0 1 1], y = -1
x = [0 0 0 0 0 0 0 1 0 1], y = -1
x = [0 0 0 0 0 0 0 1 1 0], y = -1
x = [0 0 0 0 0 0 1 0 0 0], y = 1
x = [0 0 0 0 0 0 1 0 0 1], y = -1
x = [0 0 0 0 0 0 1 0 1 0], y = -1
x = [0 0 0 0 0 0 1 0 1 1], y = 1
x = [0 0 0 0 0 0 1 1 0 0], y = -1
x = [0 0 0 0 0 0 1 1 0 1], y = 1
x = [0 0 0 0 0 0 1 1 1 0], y = 1
x = [0 0 0 0 0 0 1 1 1 1], y = -1
x = [0 0 0 0 0 1 0 0 0 0], y = 1
x = [0 0 0 0 0 1 0 0 0 1], y = -1
x = [0 0 0 0 0 1 0 0 1 0], y = -1
x = [0 0 0 0 0 1 0 0 1 1], y = 1
x = [0 0 0 0 0 1 0 1 0 0], y = -1
x = [0 0 0 0 0 1 0 1 0 1], y = 1
x = [0 0 0 0 0 1 0 1 1 0], y = 1
x = [0 0 0 0 0 1 0 1 1 1], y = -1
x = [0 0 0 0 0 1 1 0 0 0], y = -1
x = [0 0 0 0 0 1 1 0 0 1], y = 1
x = [0 0 0 0 0 1 1 0 1 0], y = 1
x = [0 0 0 0 0 1 1 0 1 1], y = -1
x = [0 0 0 0 0 1 1 1 0 0], y = 1
x = [0 0 0 0 0 1 1 1 0 1], y = -1
x = [0 0 0 0 0 1 1 1 1 1], y = 1
x = [0 0 0 0 1 0 0 0 0 0], y

In [22]:
np.random.seed(0)
num_qubits = len(X[0])
num_layers = 2
weights_init = 0.01 * np.random.randn(num_layers, (2*num_qubits) - 1, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print("Weights:", weights_init)
print("Bias: ", bias_init)

Weights: [[[ 0.01764052  0.00400157  0.00978738]
  [ 0.02240893  0.01867558 -0.00977278]
  [ 0.00950088 -0.00151357 -0.00103219]
  [ 0.00410599  0.00144044  0.01454274]
  [ 0.00761038  0.00121675  0.00443863]
  [ 0.00333674  0.01494079 -0.00205158]
  [ 0.00313068 -0.00854096 -0.0255299 ]
  [ 0.00653619  0.00864436 -0.00742165]
  [ 0.02269755 -0.01454366  0.00045759]
  [-0.00187184  0.01532779  0.01469359]
  [ 0.00154947  0.00378163 -0.00887786]
  [-0.01980796 -0.00347912  0.00156349]
  [ 0.01230291  0.0120238  -0.00387327]
  [-0.00302303 -0.01048553 -0.01420018]
  [-0.0170627   0.01950775 -0.00509652]
  [-0.00438074 -0.01252795  0.0077749 ]
  [-0.01613898 -0.0021274  -0.00895467]
  [ 0.00386902 -0.00510805 -0.01180632]
  [-0.00028182  0.00428332  0.00066517]]

 [[ 0.00302472 -0.00634322 -0.00362741]
  [-0.0067246  -0.00359553 -0.00813146]
  [-0.01726283  0.00177426 -0.00401781]
  [-0.01630198  0.00462782 -0.00907298]
  [ 0.00051945  0.00729091  0.00128983]
  [ 0.01139401 -0.01234826  0

In [23]:
opt = qml.NesterovMomentumOptimizer(0.5)
batch_size = 15

In [24]:
def accuracy(labels, predictions):
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    acc = acc / len(labels)
    return acc

In [26]:
weights = weights_init
bias = bias_init
for it in range(50):

    # Update the weights by one optimizer step, using only a limited batch of data
    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias = opt.step(cost, weights, bias, X=X_batch, Y=Y_batch)

    # Compute accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    current_cost = cost(weights, bias, X, Y)
    acc = accuracy(Y, predictions)

    print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

Iter:    1 | Cost: 2.0391365 | Accuracy: 0.5059524
Iter:    2 | Cost: 2.0063714 | Accuracy: 0.5059524
Iter:    3 | Cost: 1.5565856 | Accuracy: 0.5023810
Iter:    4 | Cost: 1.6675841 | Accuracy: 0.4976190
Iter:    5 | Cost: 1.6050198 | Accuracy: 0.4976190
Iter:    6 | Cost: 1.4930543 | Accuracy: 0.5023810
Iter:    7 | Cost: 1.4920200 | Accuracy: 0.5059524
Iter:    8 | Cost: 1.4570781 | Accuracy: 0.5059524
Iter:    9 | Cost: 1.3062366 | Accuracy: 0.5059524
Iter:   10 | Cost: 1.1250402 | Accuracy: 0.5059524
Iter:   11 | Cost: 1.1277940 | Accuracy: 0.5095238
Iter:   12 | Cost: 1.1570316 | Accuracy: 0.5107143
Iter:   13 | Cost: 1.0383128 | Accuracy: 0.5059524
Iter:   14 | Cost: 1.0637082 | Accuracy: 0.4940476
Iter:   15 | Cost: 1.0511798 | Accuracy: 0.5023810
Iter:   16 | Cost: 1.2262978 | Accuracy: 0.5059524
Iter:   17 | Cost: 1.0473645 | Accuracy: 0.5023810
Iter:   18 | Cost: 1.0541905 | Accuracy: 0.4976190
Iter:   19 | Cost: 1.0482723 | Accuracy: 0.4976190
Iter:   20 | Cost: 1.1721448 | 

In [13]:
data = np.loadtxt("data/parity_test_2.txt", dtype=int)
X_test = np.array(data[:, :-1])
Y_test = np.array(data[:, -1])
Y_test = Y_test * 2 - 1  # shift label from {0, 1} to {-1, 1}

predictions_test = [np.sign(variational_classifier(weights, bias, x)) for x in X_test]

for x,y,p in zip(X_test, Y_test, predictions_test):
    print(f"x = {x}, y = {y}, pred={p}")

acc_test = accuracy(Y_test, predictions_test)
print("Accuracy on unseen data:", acc_test)

x = [0 0 0 0 0 0 0 1 0 0], y = 1, pred=1.0
x = [0 0 0 0 0 0 0 1 1 1], y = 1, pred=-1.0
x = [0 0 0 0 0 1 1 1 1 0], y = -1, pred=1.0
x = [0 0 0 0 1 0 0 0 1 0], y = -1, pred=1.0
x = [0 0 0 0 1 0 0 0 1 1], y = 1, pred=-1.0
x = [0 0 0 0 1 0 1 0 1 1], y = -1, pred=-1.0
x = [0 0 0 0 1 0 1 1 1 1], y = 1, pred=-1.0
x = [0 0 0 0 1 1 0 0 0 1], y = 1, pred=-1.0
x = [0 0 0 0 1 1 0 0 1 0], y = 1, pred=1.0
x = [0 0 0 0 1 1 0 0 1 1], y = -1, pred=-1.0
x = [0 0 0 0 1 1 0 1 0 0], y = 1, pred=1.0
x = [0 0 0 0 1 1 1 0 0 0], y = 1, pred=1.0
x = [0 0 0 1 0 0 0 1 0 0], y = -1, pred=1.0
x = [0 0 0 1 0 0 1 0 0 1], y = 1, pred=-1.0
x = [0 0 0 1 0 0 1 0 1 0], y = 1, pred=1.0
x = [0 0 0 1 0 0 1 0 1 1], y = -1, pred=-1.0
x = [0 0 0 1 0 0 1 1 1 0], y = -1, pred=1.0
x = [0 0 0 1 0 1 0 0 0 0], y = -1, pred=1.0
x = [0 0 0 1 0 1 0 1 0 1], y = -1, pred=-1.0
x = [0 0 0 1 0 1 1 1 1 1], y = -1, pred=-1.0
x = [0 0 0 1 1 0 0 1 1 0], y = -1, pred=1.0
x = [0 0 0 1 1 1 0 0 1 0], y = -1, pred=1.0
x = [0 0 0 1 1 1 0 0 1 1], y = 1